# Chapter 3: Statistical models and inference (exercises)

All the exercises are on distance inference from parallaxes, and build on one another, so **you need to do them in order**.

The parallax of an object is its observed angular displacement with respect to a reference frame due to the movement of the observer over a baseline. Stellar parallaxes are measured using twice the Earth-Sun separation as a baseline. From geometry and the definition of the parallax, $\omega$, the distance of a star from the Sun, $r$ pc, is $1/\omega$ arcsec to a very good approximation.

But inverting a parallax to give a distance is only appropriate when we have no measurement errors. As we always have measurement errors, determining the distance given a parallax becomes an inference problem. Let us investigate how to do this

# Excercise 1: Truncated uniform prior (30 points)

Write computer programs to

* compute the posterior PDF (which is a function of distance) given the parallax, $\omega$, and its standard deviation, $\sigma_{\omega}$, using the Gaussian likelihood (equation 3.19) and an arbitrary prior (i.e., the program should be flexible to make use of different priors);
$$ P(\omega \mid r, \sigma_\omega) = \frac{1}{\sqrt{2 \pi}\sigma_\omega} 
\exp{ \left[ -\frac{1}{2\sigma_\omega^2} \left(\omega-\frac{1}{r} \right)^2 \right]} $$

* perform the empirical test described in section 3.5.4 to determine the bias and standard deviation of an arbitrary distance estimator (in particular the mode) as a function of $f_{true}$.  

Use this second program to investigate the performance of the two distance estimators given in
section 3.5.3 (i.e. the mode of the improper uniform prior) and section 3.5.5 (i.e., the mode of the
truncated uniform prior, equation 3.30). In both cases: draw simulated data from a distribution
which is uniform in the range $(0, 10^3)$ pc and zero outside this; omit from your samples simulated
parallax measurements which are negative. Specifically,

1. plot the posterior PDF for the truncated uniform prior (i.e. equation 3.29) for $f = (0.1, 0.2, 0.5, 1.0)$
and $r_{lim}=10^3$ pc; (i.e. reproduce at least the left panel of Figure 3.7). **posterior normalization can be n

2. do the tests which led to Figures 3.6 and 3.8 and plot your results (i.e. reproduce these figures;
use the same axes and ranges). Note: _the distance estimator is $1/\omega$._

# Exercise 2: Truncated uniform space density prior (40 points)

Use your programs from exercise 1 to investigate the use of the truncated uniform space density prior (equation 3.27), to define the posterior PDF, and the bias and standard deviation of the mode of this as a distance estimator. Specifically, 

1. determine the posterior PDF, and then use your program to plot it with $\omega = 1/100$ arcsec, $r_{lim} = 10^3$ pc for $f = (0.1, 0.2, 0.5, 1.0)$ (overplot these on the same plot so you can best see the differences). **Recall that a posterior should be normalized**: you will have to do this numerically here (it is sufficient to approximate the area under the curve using a sequence of narrow rectangles; section 3.4.3). Plot the **normalized** posteriors on one plot, **and** the unnormalized posteriors scaled to have their modes equal to unity on another plot. Comment on how the shape of the posterior in general, and the location of the mode in particular, changes with a function of $f$ . (You may want to plot at other values of $f$ to see what is going on.) There is of course another mode at $r = r_{lim}$ imposed by the prior, but this is independent of $f$. (2 plots to produce)

2.  calculate **analytically** the value of the mode, $r_{mode}$, of this posterior (show your working) as a function of $f$ (this is not referring to the fixed mode at $r=r_{lim}$!). Plot $\omega \cdot r_{mode}$ vs. $f$. What do you notice about the solution? For what values of $f$, if any, is the mode not defined? What value of $r_{mode}$ would be sensible to use in this case (i.e., to be consistent with our prior assumptions)? Likewise, what could we/should we do if the parallax is negative?

3. use your program from exercise 1 to calculate and plot the bias and standard deviation of $r_{mode}$ as a function of $f_{true}$ over the range $f_{true}=(0,1)$, i.e.\ produce a plot similar to Figure 3.5.  To do this you need to draw random numbers from the uniform space density prior (equation 3.27). We will look at methods for drawing random numbers from arbitrary probability distributions in a later chapter. For now you can just use the following implemention of rejection sampling, given below.

Below you'll find a R and python function that will return _approximately_ `Nsamp/3` samples for the distribution truncated at `rlim`.

In [5]:
%%file rsnip.r
r.prior1 <- function(Nsamp, rlim) { # Vectorized in Nsamp
    # Use rejection sampling. Expect 1/3 * Nsamp samples to be returned
    r1 <- runif(Nsamp, min=0, max=1)
    r2 <- runif(Nsamp, min=0, max=1)
    r <- rlim*r1[which(r2<=r1^2)]
    return(r)
}

Overwriting rsnip.r


In [6]:
def prior1(Nsamp, rlim):
    # Use rejection sampling. Expect 1/3 * Nsamp samples to be returned
    r1 = np.random.uniform(0, 1, Nsamp)
    r2 = np.random.uniform(0, 1, Nsamp)
    r = rlim * r1[r2 <= r1 ** 2]
    return r    

To keep things simple when calculating the bias and standard deviation, reject samples which
have negative parallaxes or which give undefined distances. For those cases where the mode
given by your estimator (solution to part (b)) is undefined or exceeds $r_{lim}$ , what should you
use as a distance estimator? _Hint: Think about what the posterior looks like in those cases._

4. compare the performance of this distance estimator with the mode of the posterior from the truncated uniform prior (exercise 2). Which gives lower bias and standard deviation for low and high values of $f_{true}$?

# Exercise 3: Hipparcos data (25 points)

Look at the data set `hipparcos.dat`. This contains the parallax (in mas), parallax standard deviations (`errors`) $\sigma_\omega$ (in mas), and V-band apparent magnitudes of all stars in the Hipparcos catalogue. The noise model for the parallaxes is a Gaussian. We want to derive distances to these stars using the mode estimators of the two posteriors (corresponding to the two priors) we studied in the previous two questions. Let's call
them $r_{\rm uni}$ for the mode of the truncated uniform prior in $r$, and $r_{\rm usd}$ for the 
mode of the truncated uniform space density prior.

1. Read in and inspect the data: look at the ranges, plot histograms of **each** the measurements and the fractional parallax error (as in exercice 2), plot the measurements against each other. Plot $\sigma_\omega$ vs. $\omega$ and $\sigma_\omega$ $ vs. magnitudes. Think about what these tell you. Remove stars which have any missing data (indicated with `NA`). Find out how many of each of the three variables are negative, zero, infinite. Are such values valid for these variables? Be aware of the units!

2. Calculate the distance estimates $r_{\rm uni}$ and $r_{\rm usd}$ with $r_{lim}=1000$ pc. Rather than rejecting stars with negative parallaxes or which give undefined solutions of the estimator, assign them a value which is consistent with the corresponding prior (justify your choice).  Plot histograms of $r_{\rm uni}$, $r_{\rm usd}$, and $r_{\rm uni} - r_{\rm usd}$, choosing sensible bins in each case. Explain the differences that you see.  Plot also $r_{\rm uni}$ vs. $r_{\rm usd}$ and overplot the line $r_{\rm uni} = r_{\rm usd}$. Do you see systematic differences? If so, explain why these occur.

3. The apparent magnitude, $m$, is related to the absolute magnitude, $M$, by the relation $m - M = 5\log_{10}r - 5$.  For a star of given $M$, how, qualitatively, does $m$ vary with $r$? (This is important for understanding the magnitude scale.) Compute $M$ for all stars using each of the distance estimators, and plot the histogram over $M$ in each case, as well as a histogram of the differences. What does this tell you? Plot $M$ vs.\ $r$ for your two estimators. Decsribe the result. Given that $M$ is an _intrinsic_ property of a star, what does this plot tell you about the Hipparcos survey?

Read data

# Exercise 4: A better prior (5 points) 

1. What is a significant drawback of the two priors we have considered so far (uniform in $r$ and uniform space density)?
2. Can you think of a sensible prior which might avoid this? (explain)